In [1]:
from model import get_device, get_restnet152
from torch import nn, optim
from data_loader import load_split_data

import json
import torch

In [2]:
device = get_device()
model = get_restnet152()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.003)
criterion = nn.NLLLoss()

In [3]:
import PIL
import os

with open('./datainfo.json', 'r') as f:
    data = json.load(f)      

def uwu():

    epochs = 5
    steps = 0
    running_loss = 0
    print_every = 10
    train_losses, test_losses = [], []
    trainloader, testloader = load_split_data(data, .2)

    for epoch in range(epochs):
        try :
            for inputs, labels in trainloader:
                steps += 1
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                logps = model.forward(inputs)

                loss = criterion(logps, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                if steps % print_every == 0:
                    test_loss = 0
                    accuracy = 0
                    model.eval()
                    with torch.no_grad():
                        for inputs, labels in testloader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            logps = model.forward(inputs)
                            batch_loss = criterion(logps, labels)
                            test_loss += batch_loss.item()
                            
                            ps = torch.exp(logps)
                            top_p, top_class = ps.topk(1, dim=1)
                            equals = top_class == labels.view(*top_class.shape)
                            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                    train_losses.append(running_loss/len(trainloader))
                    test_losses.append(test_loss/len(testloader))                    
                    print(f"Epoch {epoch+1}/{epochs}.. "
                        f"Train loss: {running_loss/print_every:.3f}.. "
                        f"Test loss: {test_loss/len(testloader):.3f}.. "
                        f"Test accuracy: {accuracy/len(testloader):.3f}")
                    running_loss = 0
                    model.train()
        except PIL.UnidentifiedImageError as error:
            print(error)
            er = str(error).split("'")
            os.remove(er[1])
            uwu()
        
uwu()

Calculando pesos de train, tamaño del dataset 41574
Calculando repeticiones por clase
Repeticiones calculadas, tiempo: 0.006005048751831055
Calculando los pesos por clase
Pasando los pesos para cada imagen
Pesos pasados, tiempo: 0.008000850677490234
Calculando pesos de test, tamaño del dataset 10393 
Calculando repeticiones por clase
Repeticiones calculadas, tiempo: 0.002991199493408203
Calculando los pesos por clase
Pasando los pesos para cada imagen
Pesos pasados, tiempo: 0.0020003318786621094
Creados los DataLoader para train y test


C:\Users\groso\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5.. Train loss: 2.438.. Test loss: 0.884.. Test accuracy: 0.507
Epoch 1/5.. Train loss: 0.958.. Test loss: 0.623.. Test accuracy: 0.608
Epoch 1/5.. Train loss: 0.654.. Test loss: 0.577.. Test accuracy: 0.686
Epoch 1/5.. Train loss: 0.592.. Test loss: 0.529.. Test accuracy: 0.784
Epoch 1/5.. Train loss: 0.537.. Test loss: 0.522.. Test accuracy: 0.756
Epoch 1/5.. Train loss: 0.527.. Test loss: 0.470.. Test accuracy: 0.788
Epoch 1/5.. Train loss: 0.464.. Test loss: 0.453.. Test accuracy: 0.797
Epoch 1/5.. Train loss: 0.461.. Test loss: 0.429.. Test accuracy: 0.810
Epoch 1/5.. Train loss: 0.449.. Test loss: 0.436.. Test accuracy: 0.803
Epoch 1/5.. Train loss: 0.433.. Test loss: 0.390.. Test accuracy: 0.825
Epoch 1/5.. Train loss: 0.439.. Test loss: 0.392.. Test accuracy: 0.830
Epoch 1/5.. Train loss: 0.429.. Test loss: 0.433.. Test accuracy: 0.796
Epoch 1/5.. Train loss: 0.434.. Test loss: 0.385.. Test accuracy: 0.833
Epoch 1/5.. Train loss: 0.366.. Test loss: 0.389.. Test accuracy

In [ ]:
a = "cannot identify image file <_io.BufferedReader name='./No-meme\\img_0001311.jpg'>"
a.split("'")[1]